# 1. CCTV마다 새로운 폴더에 영상 옮기기

In [1]:
import shutil
import os
# 현재 폴더 위치, 옮길 위치, 옮길 파일명 리스트 
def video_folder_shift(folder_path, new_folder_path, file_list):
    for filename in file_list:
        # 현재 파일 위치
        origin_file_path=os.path.join(folder_path, filename)
        # 옮겼을 때 파일 위치
        new_file_path=os.path.join(new_folder_path, filename)
        
        shutil.move(origin_file_path, new_file_path)


# 2. 영상에서 특정 영역 추출

- 영상이 저장된 폴더명
- 자를 영역 (다각형 지점 "시계방향") 넘파이 배열
- 추출된 영상을 저장할 경로

-> 영상이 저장된 폴더안의 모든 파일을 순회하면서 지정된 영역만 추출해 새로운 영상을 만들고, 해당 영상을 지정한 경로에 저장함

In [2]:
import cv2
import numpy as np
import os
# 영상이 저장된 폴더명, 자를 영역(다각형 지점), 영역 추출된 영상을 저장할 경로
def video_ROI(folder_path, pentagon_points, result_folder, file_list=[]):

    # 결과 영상을 저장할 폴더가 없으면 생성
    os.makedirs(result_folder, exist_ok=True)

    # 대상 파일 리스트를 지정하지 않은 경우 폴더의 모든 파일을 file_list로 저장
    if(len(file_list)==0):
        print("폴더 내 모든 파일 변환")
        for filename in os.listdir(folder_path):
            file_list.append(filename)

    for filename in file_list:
        video_path=os.path.join(folder_path,filename)
        cap = cv2.VideoCapture(video_path)

        # 원본 영상 가로, 세로 범위 저장
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))

        # 원본 영상 프레임 속도 저장
        original_fps = int(cap.get(cv2.CAP_PROP_FPS))

        # 지정한 영역 포인트를 open cv로 넣기 위해 3차원 배열로 변환함
        pentagon_points = pentagon_points.reshape((-1, 1, 2))
        
        # 결과 영상 codec
        fourcc=cv2.VideoWriter_fourcc(*'mp4v')
        # 결과 영상 저장 경로
        output_path=os.path.join(result_folder, filename)
        out = cv2.VideoWriter(output_path, fourcc, original_fps, (frame_width, frame_height))
        while True:
            ret, frame = cap.read()

            if not ret:
                break

            # 자를 영역 mask 생성
            mask = np.zeros_like(frame, dtype=np.uint8)
            cv2.fillPoly(mask, [pentagon_points], (255, 255, 255))

            # Generate the result by combining the frame and mask
            result = cv2.bitwise_and(frame, mask)

            # 결과 영상에 잘린 영역 덮어쓰기
            out.write(result)

            # 결과물 출력
            # cv2.imshow('Processed Frame', result)

            # 루프 종료 (q누르면)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # 비디오 객체 종료
        cap.release()
        out.release()

        # 모든 cv2 창 제거
    cv2.destroyAllWindows()


# 3. ROI 영역 확대

In [3]:
import cv2
import numpy as np
import os
# 영상이 저장된 폴더명, 자를 영역(다각형 지점), 영역 추출된 영상을 저장할 경로
def Enlarge_ROI(folder_path, pentagon_points, result_folder, file_list=[]):
    
    x = np.min(pentagon_points[:, 0])
    y = np.min(pentagon_points[:, 1])
    width = np.max(pentagon_points[:, 0]) - x
    height = np.max(pentagon_points[:, 1]) - y

    # 결과 영상을 저장할 폴더가 없으면 생성
    os.makedirs(result_folder, exist_ok=False)
    print("폴더 생성")
    # 대상 파일 리스트를 지정하지 않은 경우 폴더의 모든 파일을 file_list로 저장
    if(len(file_list)==0):
        print("폴더내의 모든 파일 변환")
        for filename in os.listdir(folder_path):
            file_list.append(filename)
    
    for filename in file_list:
        video_path=os.path.join(folder_path,filename)
        cap = cv2.VideoCapture(video_path)

        frame_rate = cap.get(cv2.CAP_PROP_FPS)
        output_path=os.path.join(result_folder, filename)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, frame_rate, (width, height))

        # 영상 프레임 읽어와서 처리하기
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # 영상에서 ROI 자르기
            cropped_frame = frame[y:y+height, x:x+width]
            
            # 자른 프레임 저장
            out.write(cropped_frame)
            
            # 자른 프레임 화면에 표시하기 (optional)
            # cv2.imshow('Cropped Frame', cropped_frame)
            
            # 'q' 키를 누르면 종료
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # 사용한 리소스 해제
        cap.release()
        out.release()
        cv2.destroyAllWindows()
